# Transformers 模型量化技术：GPTQ

![image](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/159_autogptq_transformers/thumbnail.jpg)

2022年，Frantar等人发表了论文 [GPTQ：Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323)。

这篇论文详细介绍了一种训练后量化算法，适用于所有通用的预训练 Transformer模型，同时只有微小的性能下降。

GPTQ算法需要通过对量化模型进行推理来校准模型的量化权重。详细的量化算法在[原始论文](https://arxiv.org/pdf/2210.17323.pdf)中有描述。

基于[`auto-gptq`](https://github.com/PanQiWei/AutoGPTQ.git) 开源实现库，transformers 支持使用GPTQ算法量化的模型。

## 使用 GPTQ 量化模型

为了使用 `auto-gptq` 库量化一个模型，我们需要向量化器传递一个数据集。

通常有两种方式构造数据集：
- 量化器支持的默认数据集（包括`['wikitext2','c4','c4-new','ptb','ptb-new']`）
- 一个字符串列表（这些字符串将被用作数据集）

### 使用支持的默认数据集来量化

在下面的示例中，让我们尝试使用`"c4"`数据集将模型量化为4位精度。支持的精度有`[2, 4, 6, 8]`。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving requirements.txt to requirements.txt
User uploaded file "requirements.txt" with length 200 bytes


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 6

In [ ]:
!pip install accelerate

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_id = "facebook/opt-6.7b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="ptb",
     desc_act=False,
)

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
quant_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

In [16]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [17]:
!nvidia-smi

Sun Dec 31 00:51:23 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              51W / 400W |  11365MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

#### 检查量化模型正确性

通过检查`线性层的属性`来确保模型已正确量化，它们应该包含`qweight`和`qzeros`属性，这些属性应该是`torch.int32`数据类型。

In [24]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('qweight',
               tensor([[-1516590485,  2022282410, -1182414405,  ..., -1537760937,
                          968190105, -1218869078],
                       [-1973762678, -1990888091,  1451669112,  ...,  1767327094,
                         1485273242, -1769109111],
                       [ -890521657,  1705355194,  2042256023,  ...,  1401452921,
                         -963081656, -1212573545],
                       ...,
                       [-1193658501,   680405620,  1700247670,  ...,  1699380954,
                        -1499949223, -2055698279],
                       [-1485273719, -1736803692,  1775278986,  ..., -1756521653,
                         2106041543, -1986435180],
                       [ 1564919956,  1151064006, -1735993498,  ..., -1317428394,
                        -1467657591, -1199920805]], device='cuda:0', dtype=torch.int32)),
              ('qzeros',
               tensor(

#### 使用 GPU 加载模型并生成文本

In [25]:
# tokenizer = AutoTokenizer.from_pretrained(model_id)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to hear that you're doing well. I'm sure you'll be able to get back to your old self soon.
Thank you! I'm hoping so.


### 使用自定义数据集量化模型（灵活可扩展，前提是准备好数据）

下面演示通过传递自定义数据集来量化一个模型。

通过字符串列表来自定义一个数据集，建议样本数不少于128（样本数太少会影响模型性能）

In [20]:
from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer

model_id = "facebook/opt-6.7b"

quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
quant_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

相比使用默认数据集，未经精心准备的自定义数据集会明显降低模型性能

In [21]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to be. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to. to.


## Homework：使用 GPTQ 量化 OPT-6.7B 模型